<a href="https://colab.research.google.com/github/sohvun/2024AID/blob/huisoo/step1/plate_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 차량 번호판 인식


## 0. 설치

### 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### opencv 설치

In [ ]:
!pip install opencv-python-headless

### yolo 설치

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

--2024-06-01 19:43:06--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  35.8MB/s    in 6.6s    

2024-06-01 19:43:13 (36.0 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2024-06-01 19:43:13--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2

### easyocr 설치

In [ ]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 20.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

### pytorch(cuda) 설치

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


### 라이브러리 임포트

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import easyocr
import re

### Is CUDA available?

In [ ]:
print("CUDA available:", torch.cuda.is_available())

## 1. 차량 추출

### 객체 탐지 준비

In [ ]:
# 클래스 이름 로드
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# 네트워크 로드
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

### 객체 탐지

In [ ]:
def process_image(image):
    height, width = image.shape[:2]

    # 블롭 생성
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # 레이어 이름 가져오기
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    # 추론 실행
    outs = net.forward(output_layers)

    return outs, height, width

### 객체 정보 추출

In [ ]:
def detect_object(outs, width, height):
    # 박스 정보 초기화
    class_ids = []
    confidences = []
    boxes = []

    # 탐지 결과 분석
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # 객체 탐지
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # 노이즈 제거
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    return boxes, confidences, class_ids, indexes

### 차량 식별

In [ ]:
def detect_car(boxes, confidences, class_ids, indexes):
    # 탐지된 차량 영역 잘라내기 및 신뢰도 점수 저장
    detected_car_images = []
    car_confidences = []
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            if label == 'car':  # 차만 탐지
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image, f'{label} {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                # 차량 영역 잘라내기
                car_img = image[y:y+h, x:x+w]
                detected_car_images.append(car_img)
                car_confidences.append(confidence)

    return detected_car_images, car_confidences

### 차량 선택

In [ ]:
def select_car(detected_car_images, car_confidences):
    # 신뢰도 점수가 가장 높은 차량 선택
    max_confidence_idx = np.argmax(car_confidences)
    best_car_img = detected_car_images[max_confidence_idx]

    # # 가장 높은 신뢰도 점수를 가진 차량 이미지 출력
    # plt.figure(figsize=(10, 10))
    # plt.imshow(cv2.cvtColor(best_car_img, cv2.COLOR_BGR2RGB))
    # plt.title(image_file)
    # plt.axis('off')
    # plt.show()

    return best_car_img

## 2. 번호판 추출

### 이미지 정보 추출

In [ ]:
def show_image(best_car_img):
  height, width, channel = best_car_img.shape

  # plt.figure(figsize=(12, 10))
  # plt.imshow(best_car_img, cmap='gray')
  # plt.show()

  return height, width, channel

### 그레이스케일 변환

In [ ]:
def convert_to_gray(best_car_img):
    gray = cv2.cvtColor(best_car_img, cv2.COLOR_BGR2GRAY)

    # plt.figure(figsize=(12, 10))
    # plt.imshow(gray, cmap='gray')
    # plt.show()

    return gray

### 가우시안 블러 적용

In [ ]:
def blur_image(gray):
  img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

  # plt.figure(figsize=(12,10))
  # plt.imshow(img_blurred, cmap='gray')
  # plt.show()

  return img_blurred

### 이미지 이진화

In [ ]:
def threshold_image(img_blurred):
  img_thresh = cv2.adaptiveThreshold(
      img_blurred,
      maxValue=255.0,
      adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
      thresholdType=cv2.THRESH_BINARY_INV,
      blockSize=19,
      C=9
  )

  # plt.figure(figsize=(12, 10))
  # plt.imshow(img_thresh, cmap='gray')
  # plt.show()

  return img_thresh

### 외곽선(객체 경계) 찾기

In [ ]:
def find_contours(img_thresh, height, width, channel):
  contours, _ = cv2.findContours(
      img_thresh,
      mode=cv2.RETR_LIST,
      method=cv2.CHAIN_APPROX_SIMPLE
  )

  temp_result = np.zeros((height, width, channel), dtype=np.uint8)

  cv2.drawContours(temp_result, contours, -1, color=(255, 255, 255))

  # plt.figure(figsize=(12, 10))
  # plt.imshow(temp_result)
  # plt.show()

  return contours, temp_result

### 외곽선 정보 추출

In [ ]:
def visualize_contours(contours, temp_result):
  contours_dict = []

  for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)
      cv2.rectangle(temp_result, pt1=(x, y), pt2=(x+w, y+h), color=(255, 255, 255), thickness=2)

      contours_dict.append({
          'contour': contour,
          'x': x,
          'y': y,
          'w': w,
          'h': h,
          'cx': x + (w / 2),
          'cy': y + (h / 2)
      })

  # plt.figure(figsize=(12, 10))
  # plt.imshow(temp_result, cmap='gray')
  # plt.show()

  return contours_dict

### 외곽선 필터링

In [ ]:
MIN_AREA = 80
MIN_WIDTH, MIN_HEIGHT = 2, 8
MIN_RATIO, MAX_RATIO = 0.25, 1.0

def draw_bounding_boxes(contours_dict, height, width, channel):
  possible_contours = []

  cnt = 0
  for d in contours_dict:
      area = d['w'] * d['h']
      ratio = d['w'] / d['h']

      if area > MIN_AREA \
      and d['w'] > MIN_WIDTH and d['h'] > MIN_HEIGHT \
      and MIN_RATIO < ratio < MAX_RATIO:
          d['idx'] = cnt
          cnt += 1
          possible_contours.append(d)

  # visualize possible contours
  temp_result = np.zeros((height, width, channel), dtype=np.uint8)

  for d in possible_contours:
      # cv2.drawContours(temp_result, d['contour'], -1, (255, 255, 255))
      cv2.rectangle(temp_result, pt1=(d['x'], d['y']), pt2=(d['x']+d['w'], d['y']+d['h']), color=(255, 255, 255), thickness=2)

  # plt.figure(figsize=(12, 10))
  # plt.imshow(temp_result, cmap='gray')
  # plt.show()

  return possible_contours

### 외곽선 매칭

In [ ]:
MAX_DIAG_MULTIPLYER = 5 # 5
MAX_ANGLE_DIFF = 12.0 # 12.0
MAX_AREA_DIFF = 0.5 # 0.5
MAX_WIDTH_DIFF = 0.8
MAX_HEIGHT_DIFF = 0.2
MIN_N_MATCHED = 5 # 3

def find_matching_contours(possible_contours, height, width, channel):
  def find_chars(contour_list):
      matched_result_idx = []

      for d1 in contour_list:
          matched_contours_idx = []
          for d2 in contour_list:
              if d1['idx'] == d2['idx']:
                  continue

              dx = abs(d1['cx'] - d2['cx'])
              dy = abs(d1['cy'] - d2['cy'])

              diagonal_length1 = np.sqrt(d1['w'] ** 2 + d1['h'] ** 2)

              distance = np.linalg.norm(np.array([d1['cx'], d1['cy']]) - np.array([d2['cx'], d2['cy']]))
              if dx == 0:
                  angle_diff = 90
              else:
                  angle_diff = np.degrees(np.arctan(dy / dx))
              area_diff = abs(d1['w'] * d1['h'] - d2['w'] * d2['h']) / (d1['w'] * d1['h'])
              width_diff = abs(d1['w'] - d2['w']) / d1['w']
              height_diff = abs(d1['h'] - d2['h']) / d1['h']

              if distance < diagonal_length1 * MAX_DIAG_MULTIPLYER \
              and angle_diff < MAX_ANGLE_DIFF and area_diff < MAX_AREA_DIFF \
              and width_diff < MAX_WIDTH_DIFF and height_diff < MAX_HEIGHT_DIFF:
                  matched_contours_idx.append(d2['idx'])

          # append this contour
          matched_contours_idx.append(d1['idx'])

          if len(matched_contours_idx) < MIN_N_MATCHED:
              continue

          matched_result_idx.append(matched_contours_idx)

          unmatched_contour_idx = []
          for d4 in contour_list:
              if d4['idx'] not in matched_contours_idx:
                  unmatched_contour_idx.append(d4['idx'])

          unmatched_contour = np.take(possible_contours, unmatched_contour_idx)

          # recursive
          recursive_contour_list = find_chars(unmatched_contour)

          for idx in recursive_contour_list:
              matched_result_idx.append(idx)

          break

      return matched_result_idx

  result_idx = find_chars(possible_contours)

  matched_result = []
  for idx_list in result_idx:
      matched_result.append(np.take(possible_contours, idx_list))

  # visualize possible contours
  temp_result = np.zeros((height, width, channel), dtype=np.uint8)

  for r in matched_result:
      for d in r:
  #         cv2.drawContours(temp_result, d['contour'], -1, (255, 255, 255))
          cv2.rectangle(temp_result, pt1=(d['x'], d['y']), pt2=(d['x']+d['w'], d['y']+d['h']), color=(255, 255, 255), thickness=2)

  # plt.figure(figsize=(12, 10))
  # plt.imshow(temp_result, cmap='gray')
  # plt.show()

  return matched_result

### 번호판 선택

In [ ]:
PLATE_WIDTH_PADDING = 1.3 # 1.3
PLATE_HEIGHT_PADDING = 1.5 # 1.5
MIN_PLATE_RATIO = 3
MAX_PLATE_RATIO = 10

def extract_plate_images(filtered_matched_result, img_thresh, width, height):
  plate_imgs = []
  plate_infos = []

  for i, matched_chars in enumerate(filtered_matched_result):
      sorted_chars = sorted(matched_chars, key=lambda x: x['cx'])

      plate_cx = (sorted_chars[0]['cx'] + sorted_chars[-1]['cx']) / 2
      plate_cy = (sorted_chars[0]['cy'] + sorted_chars[-1]['cy']) / 2

      plate_width = (sorted_chars[-1]['x'] + sorted_chars[-1]['w'] - sorted_chars[0]['x']) * PLATE_WIDTH_PADDING

      sum_height = 0
      for d in sorted_chars:
          sum_height += d['h']

      plate_height = int(sum_height / len(sorted_chars) * PLATE_HEIGHT_PADDING)

      triangle_height = sorted_chars[-1]['cy'] - sorted_chars[0]['cy']
      triangle_hypotenus = np.linalg.norm(
          np.array([sorted_chars[0]['cx'], sorted_chars[0]['cy']]) -
          np.array([sorted_chars[-1]['cx'], sorted_chars[-1]['cy']])
      )

      angle = np.degrees(np.arcsin(triangle_height / triangle_hypotenus))

      rotation_matrix = cv2.getRotationMatrix2D(center=(plate_cx, plate_cy), angle=angle, scale=1.0)

      img_rotated = cv2.warpAffine(img_thresh, M=rotation_matrix, dsize=(width, height))

      img_cropped = cv2.getRectSubPix(
          img_rotated,
          patchSize=(int(plate_width), int(plate_height)),
          center=(int(plate_cx), int(plate_cy))
      )

      if img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO or img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO > MAX_PLATE_RATIO:
          continue

      plate_imgs.append(img_cropped)
      plate_infos.append({
          'x': int(plate_cx - plate_width / 2),
          'y': int(plate_cy - plate_height / 2),
          'w': int(plate_width),
          'h': int(plate_height)
      })

      # plt.subplot(len(filtered_matched_result), 1, i+1)
      # plt.imshow(img_cropped, cmap='gray')
      # plt.show()

  return plate_imgs

## 3. 번호판 인식

In [ ]:
def extract_plate_characters(plate_imgs):
  longest_idx, longest_text = -1, 0
  plate_chars = []

  for i, plate_img in enumerate(plate_imgs):
      plate_img = cv2.resize(plate_img, dsize=(0, 0), fx=1.2, fy=1.2)
      _, plate_img = cv2.threshold(plate_img, thresh=0.0, maxval=255.0, type=cv2.THRESH_BINARY | cv2.THRESH_OTSU)

      _, thresh = cv2.threshold(plate_img, 240, 255, cv2.THRESH_BINARY_INV)

      # Contour Detection 수행
      contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      # 채워진 원 제거를 위한 마스크 생성
      mask = np.ones_like(plate_img) * 255

      # Contour를 순회하며 채워진 원을 마스크에 그림
      for contour in contours:
          area = cv2.contourArea(contour)
          perimeter = cv2.arcLength(contour, True)
          # Contour를 근사화하여 꼭지점 수 구하기
          approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
          # 채워진 원으로 판별되는 Contour의 내부를 마스크에 채움
          if len(approx) > 10:
              cv2.drawContours(mask, [contour], -1, 0, thickness=cv2.FILLED)

      # 마스크를 사용하여 이미지에서 채워진 원을 제거
      plate_img = cv2.bitwise_and(plate_img, plate_img, mask=mask)

      plate_min_x, plate_min_y = plate_img.shape[1], plate_img.shape[0]
      plate_max_x, plate_max_y = 0, 0

      for contour in contours:
          x, y, w, h = cv2.boundingRect(contour)

          area = w * h
          ratio = w / h

          if area > MIN_AREA \
          and w > MIN_WIDTH and h > MIN_HEIGHT \
          and MIN_RATIO < ratio < MAX_RATIO:
              if x < plate_min_x:
                  plate_min_x = x
              if y < plate_min_y:
                  plate_min_y = y
              if x + w > plate_max_x:
                  plate_max_x = x + w
              if y + h > plate_max_y:
                  plate_max_y = y + h

      img_result = plate_img[plate_min_y:plate_max_y, plate_min_x:plate_max_x]

      # 필요한 후속 처리 진행
      img_result = plate_img.copy()
      img_result = cv2.GaussianBlur(img_result, ksize=(1, 1), sigmaX=0)
      _, img_result = cv2.threshold(img_result, thresh=0.0, maxval=255.0, type=cv2.THRESH_BINARY | cv2.THRESH_OTSU)
      img_result = cv2.copyMakeBorder(img_result, top=10, bottom=10, left=10, right=10, borderType=cv2.BORDER_CONSTANT, value=(0,0,0))

      reader = easyocr.Reader(['ko'], gpu=torch.cuda.is_available())
      try:
        chars = reader.readtext(img_result, detail=0)[0]
      except IndexError:
        chars = ""

      # 결과 필터링: 형식에 맞는 텍스트 추출
      result_chars = ''
      has_digit = False
      # 정규식 패턴: 2-3개의 숫자, 1개의 한글, 4개의 숫자
      pattern = re.compile(r'\d{2,3}[가-힣]\d{4}')
      for c in chars:
          if ord('가') <= ord(c) <= ord('힣') or c.isdigit():
            if c.isdigit():
                  has_digit = True
            result_chars += c
      # 정규식 패턴으로 필터링
      result_chars = pattern.findall(result_chars)
      # 결과 출력
      plate_chars.append(result_chars)

      if has_digit and len(result_chars) > longest_text:
          longest_idx = i

      # plt.subplot(len(plate_imgs), 1, i+1)
      # plt.imshow(img_result, cmap='gray')
      # plt.show()

      # print(result_chars)

      return result_chars

## 4. 전체 과정

### 이미지 폴더 적용

In [ ]:
# plate 폴더 경로
plate_folder = '/content/drive/MyDrive/AID/plate'

# plate 폴더 내의 하위 폴더들을 리스트로 저장
subfolders = [os.path.join(plate_folder, subfolder) for subfolder in os.listdir(plate_folder)]
subfolders = sorted(subfolders)

plate_list = ['103우2951', '36더0252', '55소6637']
plate_list = [[plate] for plate in plate_list]
sum1 = [0] * 3
sum2 = [0] * 3
cnt = [0] * 3

# 각 하위 폴더를 순회하며 이미지 파일 처리
for i, subfolder in enumerate(subfolders):
    image_files = os.listdir(subfolder)
    sum1[i] = len(image_files)
    cnt[i] = 0
    for image_file in image_files:
        image_path = os.path.join(subfolder, image_file)
        image = cv2.imread(image_path)

        outs, height, width = process_image(image)
        boxes, confidences, class_ids, indexes = detect_object(outs, width, height)
        detected_car_images, car_confidences = detect_car(boxes, confidences, class_ids, indexes)
        best_car_img = select_car(detected_car_images, car_confidences)

        height, width, channel = show_image(best_car_img)

        gray = convert_to_gray(best_car_img)

        img_blurred = blur_image(gray)

        img_thresh = threshold_image(img_blurred)

        contours, temp_result = find_contours(img_thresh, height, width, channel)

        contours_dict = visualize_contours(contours, temp_result)

        possible_contours = draw_bounding_boxes(contours_dict, height, width, channel)

        matched_result = find_matching_contours(possible_contours, height, width, channel)

        plate_imgs = extract_plate_images(matched_result, img_thresh, width, height)

        result_chars = extract_plate_characters(plate_imgs)

        if result_chars == plate_list[i]:
          cnt[i] += 1
        if result_chars != []:
          sum2[i] += 1

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


### 정확도

#### empty 포함

In [ ]:
for i in range(len(plate_list)):
  print(plate_list[i])
  print(f'{cnt[i]}/{sum1[i]} = {cnt[i] / sum1[i]}')
print(f'-> {sum(cnt)}/{sum(sum1)} = {sum(cnt) / sum(sum1)}')

['103우2951']
8/10 = 0.8
['36더0252']
22/34 = 0.6470588235294118
['55소6637']
22/31 = 0.7096774193548387
-> 52/75 = 0.6933333333333334


#### empty 미포함

In [ ]:
for i in range(len(plate_list)):
  print(plate_list[i])
  print(f'{cnt[i]}/{sum2[i]} = {cnt[i] / sum2[i]}')
print(f'-> {sum(cnt)}/{sum(sum2)} = {sum(cnt) / sum(sum2)}')

['103우2951']
8/9 = 0.8888888888888888
['36더0252']
22/24 = 0.9166666666666666
['55소6637']
22/23 = 0.9565217391304348
-> 52/56 = 0.9285714285714286
